add lasso and ridge
#no skl learn :/

In [0]:
import numpy as np
from scipy import stats

In [0]:
class LR:
  def __init__(self  ,degree = 10, fit_by=1 , learning_rate=1 ,epsilon=1E-3 ,earlystoping= True , tol=1E-3 , patience= 5 , val=0.2 , alpha=1 , r=0 ):
    ''' function will do linear regression for data, it recive, degree (1),  and hot to fit polynimal (2), furier base or mixsed(3)
        args:
          degree - rank of polinomal / fourier base series 
          fit_by - how to do the polynomal fit 
          
    '''
    
    self.epsilon=epsilon
    self.fit_by=fit_by
    self.learning_rate=learning_rate
    self.DEGREE = degree*fit_by
#     self.cost = cost
    self.earlystoping= earlystoping
    self.tol=tol
    self.patience= patience
    self.val = val # between 0-1
    self.alpha= alpha
    self.r=r
    
    
#     self.weigth maybe we can start with set weigth and update them ?
    # we need to normilize the x values to predict
    # ymin is b in the function
   
  
  
  
  
  def normalize(self, X, y=None):
    ''' transfering the data to  0-1 range, keeping normlization value
    args:
      X-  range of fiting numpy aray
      values prdicted or to predict by- len of x
    
    atributes :
      self.miny -  min of y predicted value
      self.minx  - min x of x given
      self.maxy  - max y of predicted value
      self.maxx  - max x of prdeicted value 
      
       
    
    
    '''
    if y is not None:
      self.miny = np.min(y)
      y -= self.miny
      self.maxy = np.max(y)
      y /= self.maxy      
      self.minx = np.min(X)
      self.maxx = np.max(X)
    X -= self.minx
    
    X /= (self.maxx-self.minx)
    
    if y is not None:
      return X.reshape(-1,1), y.reshape(-1,1)
    else:
      return X.reshape(-1,1)
    
    
  def un_norm(self,X, y, ) :
    ''' transfering the data from  0-1 range to the predicted range
    args:
      X-  range of fiting numpy aray
      y-values prdicted or to predict by- len of x
    
    atributes :
     
    return: 
      X,y in original scale 
      
       
    
    
    '''
    
    y*=self.maxy + self.miny
    X*=(self.maxx + self.minx)+self.minx
    return y
  
  
  def choose_model(self,xnew):
    exp=np.arange(self.DEGREE+1)
    # use meaningful names instead of a & b like x_col and exp_col
    # Or reshape "in place"
    
    a=xnew.reshape(-1,1) #a.shape (1000, 1)


#     print('a.shape', a.shape)
    
    b=exp.reshape(1,-1)
#     print('b.shape', b.shape) b.shape (1, 11)
    
    pol=a**b
#     print( 'pol ' ,pol.shape)  pol  (1000, 11)
    f_base= np.hstack((  np.ones(len(a)).reshape(-1,1) , np.cos(2*np.pi *b*a) ,  np.sin(2*np.pi *b*a))  )
    model = np.hstack (( self.fit_by%2*pol, self.fit_by//2*f_base))
    
#     self.model =model[ ~np.all(model == 0, axis=0)] # maybe axis 1?
    # Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 6 is different from 19)
    if self.fit_by == 1:
      self.model=pol
    elif self.fit_by == 2 :
      self.model=f_base
    else:
      self.model=model
      
  
    
    
  def predict (self, x ):
#     ypred = np.matmul(self.model ,self.weigths )
    xnorm = self.normalize ( x )
    ypred=np.matmul(self.model,self.weigths)
    print(self.weigths)
    
   
    return self.un_norm(xnorm,ypred)
    
  def fit (self, X, y)   :
    
    self.weigths = np.random.uniform(-0.1,0.1, self.fit_by * self.DEGREE +1).reshape(-1,1) #(11,1)
    Xnor , ynor =self.normalize(X, y) # (1000,1)
    r= self.r
    alpha =self.alpha
    #shufle and split no sklearn  
    indices = np.arange(y.shape[0])
    np.random.shuffle(indices)

    X = Xnor[indices]
    y = ynor[indices]
    
    #split
    l=len(y)
    ii= int(self.val *l)
    x_val ,y_val = X[:ii] , y[:ii]
    x_train, y_train = X[ ii: ] , y[ii :]
    self.choose_model(x_train)
    # start with train 
    i=0
 
    w_old=self.weigths.copy()
    train_contin =True
    while True:   # we want to cuntinue until break
      i+=1
      pred = np.matmul( self.model ,self.weigths )
      delta = y_train - pred
      derr =- delta * self.model #error derivative   
      av_d = derr.mean(axis=0).reshape(-1,1)
      w= self.weigths
      self.weigths -= av_d  + r*alpha*w + (alpha*(1-r)/2)*w
      if i%2000==0:
        w_old=self.weigths
        w_new=self.weigths.copy()
      

        if np.sum(( self.weigths-w_old ) **2) < self.epsilon:  #equel to error
          train_contin = False
        else:
          w_old = self.weigths.copy()
      
      
    i=0
    self.choose_model(x_val)
    w_old=self.weigths.copy()
    train_contin =True
    patience_left = sel.patience  +1
    while (patience_left>= 0) or (train_contin) :   # we want to cuntinue until break
      patience_left-=1
      pred = np.matmul( self.model ,self.weigths )
      delta = y_val - pred
      derr =- delta * self.model #error derivative   
      av_d = derr.mean(axis=0).reshape(-1,1)
      w= self.weigths
      self.weigths -= av_d  + r*alpha*w + (alpha*(1-r)/2)*w
      mses = (delta**2).mean(axis=1)
      if patience_left==1 and mses<self.tol:
        break
      if i%2000==0:
        w_old=self.weigths
        w_new=self.weigths.copy()
    

        if np.sum(( self.weigths-w_old ) **2) < self.epsilon:  #equel to error
          train_contin = False
        else:
          w_old = self.weigths.copy()
    

  
    
    
    

In [0]:

SAMPLE_COUNT = int(1E3)
COEF = 1E-2
a = np.random.rand(SAMPLE_COUNT);
b = np.sin(COEF * np.cumsum(1 / (a - a.mean())))
c = np.cumsum(b - b.mean())
NOISE = 2E-1
y = c + NOISE * np.random.rand(SAMPLE_COUNT)
y /= np.random.rand()
y -= 1 / np.random.rand()
X = np.arange(SAMPLE_COUNT) + np.random.rand(SAMPLE_COUNT)
X /= np.random.rand() * SAMPLE_COUNT
X -= 1 / np.random.rand()

x=LR(degree = 3,fit_by=1 , learning_rate=1 ,epsilon=1E-3,earlystoping= True , tol=1E-3 , patience=5 , val=0.2 , alpha=0 , r=0.5)
x.fit(X,y)
y_pred=x.predict(X)
